Функция на вход получает список из белков и/или список из болезней и выдает граф. Дефолтный граф - полный.

# Белки и болезни

Белки: 'Microtubule-associated protein tau', 'Cholinesterase', 'Apolipoprotein A-IV', 'Collagen alpha-1(III) chain', 'Angiopoietin-related protein 3', 'Peroxiredoxin-2', 'Complement C1r subcomponent-like protein', 'Peroxiredoxin-1', 'Ras GTPase-activating protein nGAP', 'SPARC', 'Prolactin', 'Glutathione peroxidase 3', 'Tenascin-X', 'Glial fibrillary acidic protein', 'Zinc-alpha-2-glycoprotein', 'Hyaluronan-binding protein 2', 'Keratin-type II cytoskeletal 2 epidermal', 'Lactotransferrin', 'Pappalysin-1', 'Gelsolin', 'Thrombomodulin', 'Biotinidase', 'Corticosteroid-binding globulin', 'Inter-alpha-trypsin inhibitor heavy chain H4', 'Complement factor D', 'Clusterin', 'Complement component C9', 'Coagulation factor XIII A chain', 'L-selectin', 'Serotransferrin', 'Histidine-rich glycoprotein', 'Angiotensinogen', 'Mannan-binding lectin serine protease 1', 'Carbonic anhydrase 1', 'Ig mu heavy chain disease protein and Ig mu chain C', 'Extracellular matrix protein 1', 'Complement C1q subcomponent subunit C', 'Endothelial protein C receptor', 'Beta-2-microglobulin', 'Xaa-Pro dipeptidase', 'Mannan-binding lectin serine protease 2A', 'IgGFc-binding protein', 'Alpha-1-antichymotrypsin', 'Occludin', 'Myeloperoxidase', 'CD44 antigen', 'Carboxypeptidase B2', 'Serum paraoxonase/lactonase 3', 'Complement component C7', 'Plasminogen', '72 KDa type IV collagenase', 'Phospholipid transfer protein', 'Protein_deglycase_DJ-1', 'Apolipoprotein D', 'Neutrophil gelatinase-associated lipocalin', 'Resistin', 'Complement factor H', 'Fibronectin', 'Serum amyloid A-1 and A-2 proteins', 'Fibulin-1', 'Alpha-2-antiplasmin', 'Transcription factor SOX-11', 'Collagen alpha-1(I) chain', 'Actin, alpha cardiac muscle 1', 'Ceruloplasmin', 'Cation-independent mannose-6-phosphate receptor', 'Vitamin K-dependent protein S', 'Vitamin K-dependent protein C', 'B-cell scaffold protein with ankyrin repeats', 'A disintegrin and metalloproteinase with thrombospondin motifs 9', 'Complement C3', 'E-selectin', 'Sterile alpha motif domain-containing protein 9-like', 'Melanotransferrin', 'Dickkopf-related protein 1', 'Haptoglobin', 'Ig gamma-1 chain C region', 'Ig mu chain C region', 'Collagen alpha-2(I) chain', 'Interstitial collagenase', 'Target of Nesh-SH3', 'Antithrombin-III', 'cDNA FLJ53327, highly similar to Gelsolin', 'Complement C1q subcomponent subunit A', 'Serum albumin', 'Coagulation factor V', 'Metalloproteinase inhibitor 2', 'Hepatocyte growth factor-like protein', 'Collagen alpha-1(XVIII) chain', 'Matrix Gla protein', 'Cadherin-5', 'Beta-Ala-His dipeptidase', 'Coagulation factor X', 'Protein S100-A12', 'Hemopexin', 'Ficolin-3', 'Stromelysin-1', 'Pregnancy zone protein', 'Mucin-16', 'Ferritin heavy chain', 'Atrial natriuretic peptide receptor 1', 'Keratin type I cytoskeletal 9', 'Natriuretic peptides B', 'Aromatase', 'A disintegrin and metalloproteinase with thrombospondin motifs 2', 'Proteoglycan 4', 'Coagulation factor XIII B chain', 'Calcitonin gene-related peptide 1', 'Thrombospondin-1', '78 kDa glucose-regulated protein', 'Fatty acid-binding protein heart', 'Adiponectin', 'Fetuin-B', 'Protein Z-dependent protease inhibitor', 'Gamma-enolase', 'Heat shock protein beta-1', 'Platelet endothelial cell adhesion molecule', 'Epidermal growth factor receptor', 'Vascular cell adhesion protein 1', 'Complement C2', 'Myeloblastin', 'Tissue-type plasminogen activator', 'Protein S100-B', 'Keratin type I cytoskeletal 10', 'Alpha-1-acid glycoprotein 1', 'Myelin basic protein', 'Coagulation factor XI', 'Alpha-1-antitrypsin', 'Attractin', 'Matrix metalloproteinase-9', 'Calcitonin', 'Sex hormone-binding globulin', 'von Willebrand factor', 'Complement component C8 beta chain', 'Apolipoprotein C-III', 'Follistatin-related protein 1', 'Chromogranin-A', 'P-selectin', 'Apolipoprotein C-I', 'Cholesteryl ester transfer protein', 'Coagulation factor XII', 'Complement factor B', 'Apolipoprotein L1', 'Osteopontin', 'Complement  C4', 'C-reactive protein', 'Ficolin-2', 'Metalloproteinase inhibitor 4', 'Complement factor I', 'Desmoplakin', 'Ferritin light chain', 'Apolipoprotein A-I', 'Protein AMBP', 'Elastin', 'Vitronectin', 'Apolipoprotein F', 'CD5 antigen-like', 'Apolipoprotein A-II', 'ADM', 'Apolipoprotein C-II', 'Creatine kinase B-type', 'Dickkopf-related protein 2', 'Insulin-like growth factor-binding protein 2', 'Claudin-5', 'Complement C1q subcomponent subunit B', 'Plasma serine protease inhibitor', 'Insulin-like growth factor-binding protein 3', 'Intercellular adhesion molecule 1', 'Vascular endothelial growth factor B', 'Nucleoside diphosphate kinase A', 'Complement C5', 'Afamin', 'Tissue_factor_pathway_inhibitor', 'Complement component C8 alpha chain', 'Interleukin-10', 'A disintegrin and metalloproteinase with thrombospondin motifs 20', 'Protein S100-A9', 'N(G),N(G)-dimethylarginine dimethylaminohydrolase 1', 'Leucine-rich alpha-2-glycoprotein', 'Metalloproteinase inhibitor 1', 'Glutathione S-transferase P', 'Platelet-activating factor acetylhydrolase', 'Kininogen-1', 'Transferrin receptor protein 1', 'Cystatin-C', 'Tenascin', 'Coagulation factor IX', 'Plasminogen activator inhibitor 1', 'Vascular endothelial growth factor D', 'Insulin-like growth factor-binding protein complex acid labile subunit', 'Mannose-binding protein C', 'Beta-nerve growth factor', 'Fibrinogen alpha chain', 'Fructose-bisphosphate aldolase B', 'Complement C1s subcomponent', 'Tumor necrosis factor receptor superfamily member 1A', 'Galectin-3-binding protein', 'Plasma protease C1 inhibitor', 'Complement component C6', 'Serum amyloid P-component', 'Plastin-2', 'Fibrinogen gamma chain', 'Apolipoprotein B-100', 'Alpha-2-HS-glycoprotein', 'Retinol-binding protein 4', 'Vascular non-inflammatory molecule 3', 'Neuropilin-2', 'Vasorin', 'CD40 ligand', 'Thyroxine-binding globulin', 'Insulin-like growth factor-binding protein 1', 'Glutamate receptor ionotropic NMDA 2B', 'Coagulation factor VIII', 'Alpha-1B-glycoprotein_VAR_018369', 'Phosphatidylcholine-sterol acyltransferase', '60 kDa heat shock protein mitochondrial', 'Autism susceptibility gene 2 protein', 'Inter-alpha-trypsin inhibitor heavy chain H1', 'Cadherin-13', 'Insulin-like growth factor I', 'Hemoglobin subunit alpha', 'Transthyretin', 'Coagulation factor VII', 'Oxidized low-density lipoprotein receptor 1', 'Lumican', 'Serum paraoxonase/arylesterase 1', 'Lipopolysaccharide-binding protein', 'Apolipoprotein(a)', 'Pigment epithelium-derived factor', 'Apolipoprotein E', 'Heparin cofactor 2', 'Actin, aortic smooth muscle', 'Fibrinogen beta chain', 'Kallistatin', 'Carboxypeptidase N catalytic chain', 'Lysozyme C', 'Glutamate receptor ionotropic NMDA 2A', 'Interleukin-6', 'Alpha-2-macroglobulin', 'Thyroglobulin', 'Tumor necrosis factor receptor superfamily member 1B', 'Galectin-3', 'Proenkephalin-A', 'Creatine kinase M-type', 'Spermine oxidase', 'Adipocyte plasma membrane-associated protein', 'Apolipoprotein C-IV', 'Angiogenin', 'Vitamin D-binding protein', 'Prothrombin', 'Beta-2-glycoprotein 1'

Болезни:
'Болезни кожи и соединительной ткани', 'Новообразования', 'Заболевания нервной системы', 'Профессиональные болезни', 'Болезни кровеносной и лимфатической системы', 'Болезни животных', 'Врожденные,\xa0наследственные болезни и аномалии. Болезни раннего детского возраста', 'Болезни сердечно-сосудистой системы', 'Глазные болезни', 'Заболевания дыхательной системы', 'Урологические\xa0заболевания\xa0и\xa0болезни\xa0мужских половых\xa0органов', 'Заболевания опорно-двигательной системы', 'Оториноларингологические заболевания ', 'Бактериальный инфекции и микозы', 'Заболевания стоматогнотической системы', 'Заболевания пищеварительной системы', 'Болезни нарушения питания и обмена веществ', 'Болезни эндокринной системы', 'Патологические состояния, признаки и симптомы', 'Болезни имунной системы', 'Поведение и механизмы поведения', 'Раны и травмы', 'Болезни\xa0женских половых\xa0органов и\xa0осложнения беременности', 'Заболевания вследствие воздействия химических веществ', 'Психические расстройства'

In [6]:
my_proteins = list(set(tb2.Protein.sample(1)))
my_diseases = list(set(tb2.disease.sample(50)))

In [7]:
tb2

,Protein,geneSymbol,diseaseName,diseaseClass,disease
0,Alpha-1-acid glycoprotein 1,ORM1,Drug toxicity,C25,Заболевания вследствие воздействия химических ...
1,Alpha-1-acid glycoprotein 1,ORM1,Adverse reaction to drug,C25,Заболевания вследствие воздействия химических ...
2,Alpha-1-antitrypsin,SERPINA1,Drug toxicity,C25,Заболевания вследствие воздействия химических ...
3,Alpha-1-antitrypsin,SERPINA1,Adverse reaction to drug,C25,Заболевания вследствие воздействия химических ...
4,Alpha-2-HS-glycoprotein,AHSG,Drug toxicity,C25,Заболевания вследствие воздействия химических ...
...,...,...,...,...,...
10671,Collagen alpha-1(I) chain,COL3A1,Vertebral Artery Dissection,C26,Раны и травмы
10672,Matrix metalloproteinase-9,MMP9,Aortic Rupture,C26,Раны и травмы
10673,Matrix metalloproteinase-9,MMP9,"Aortic Aneurysm, Ruptured",C26,Раны и травмы
10674,Tenascin,TNC,Lung Injury,C26,Раны и травмы


### Интерактивный граф #1 

In [9]:
plot_bgraph(proteins = ["Tenascin"], diseases = ["Drug toxicity", "Поведение и механизмы поведения"], depth = 1)

Loading BokehJS ...

### Интерактивный граф #2 

In [61]:
plot_cgraph(proteins = ["Alpha-2-HS-glycoprotein"], diseases = ["Vertebral Artery Dissection", "Раны и травмы"], depth = 0)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'font-famil…

## Подготовка 

In [1]:
# загрузка необходимых таблиц

import pandas as pd

tb2 = pd.read_csv("C:/Users/a120/MRM/tb2.tsv", sep = "\t")
edges_df = pd.read_csv("C:/Users/a120/MRM/edges_df.tsv", sep = "\t")
dis = pd.read_csv("C:/Users/a120/MRM/diseases.tsv", sep = "\t")
#nodes_df = pd.read_csv("C:/Users/a120/MRM/nodes_df.tsv", sep = "\t")

In [2]:
def concat(*args):
    return pd.concat([x for x in args if not x.empty])

In [3]:
# загрузка необходимых пакетов

import pandas as pd
import networkx
import matplotlib.pyplot as plt
import numpy as np
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.plotting import output_file
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap

import ipycytoscape
import ipywidgets
import json

## Функция для bokeh

In [4]:
def plot_bgraph(table = tb2, proteins = None, diseases = None, depth = 0):
    # функция строит граф ассоциации болезнь-белок
    # на вход подается таблица, список белков и список болезней
    # список белков и список болезней - в формате списка []
    # аргумент deep показывает, до какой "глубины" нужно рисовать граф:
    # только про классы или про болезни вообще
    # depth = 0 --> только про классы (если изначально как diseases не введена болезнь)
    # depth = 1 --> про болезни
    
    diseases_list = []
    classes_list = []
    table1 = pd.DataFrame()
    table2 = pd.DataFrame()
    table3 = pd.DataFrame()
    
    for item in diseases:
        if item in list(dis.diseaseName):
            diseases_list.append(item)
        if item in list(dis.disease):
            classes_list.append(item)
    
    if depth == 0:
        if proteins != None:
            proteins = pd.DataFrame(proteins)
            proteins.columns = ["Protein"]
            table1 = table.merge(proteins, on = "Protein")
        
        if len(diseases_list) != 0:
            diseases_list = pd.DataFrame(diseases_list)
            diseases_list.columns = ["diseaseName"]
            table2 = table.merge(diseases_list, on = "diseaseName")
            
        if len(classes_list) != 0:
            classes_list = pd.DataFrame(classes_list)
            classes_list.columns = ["disease"]
            table3 = table.merge(classes_list, on = "disease")

        table = concat(*[table1, table2, table3])
        table = table.drop_duplicates()
        # собственно, создаем граф    
        G = networkx.from_pandas_edgelist(table, "Protein", "disease")
            
    elif depth == 1:
        if proteins != None:
            proteins = pd.DataFrame(proteins)
            proteins.columns = ["Protein"]
            table1 = tb2.merge(proteins, on = "Protein")
        if len(diseases_list) != 0:
            diseases_list = pd.DataFrame(diseases_list)
            diseases_list.columns = ["diseaseName"]
            table2 = tb2.merge(diseases_list, on = "diseaseName")
        if len(classes_list) != 0:
            classes_list = pd.DataFrame(classes_list)
            classes_list.columns = ["disease"]
            table3 = tb2.merge(classes_list, on = "disease")
        table = concat(*[table1, table2, table3])
        table = table.drop_duplicates()
        # собственно, создаем граф
        G = networkx.from_pandas_edgelist(table, "Protein", "diseaseName")
    
    # аутпут в ноутбук
    output_notebook()

    # подгонка размера и цвета
    degrees = dict(networkx.degree(G))
    networkx.set_node_attributes(G, name='degree', values=degrees)
    number_to_adjust_by = 5
    adjusted_node_size = dict([(node, 0.3*degree+number_to_adjust_by) for node, degree in networkx.degree(G)])
    networkx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)
    size_by_this_attribute = 'adjusted_node_size'
    color_by_this_attribute = 'adjusted_node_size'

    # Blues8, Reds8, Purples8, Oranges8, Viridis8
    color_palette = Reds8

    title = 'Protein-disease associations'

    # hovering
    HOVER_TOOLTIPS = [("Type", "@index"), ("Degree", "@degree")]

    plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

    # network graph object
    network_graph = from_networkx(G, networkx.spring_layout, scale=10, center=(0, 0))

    # node sizes and colors according to node degree
    minimum_value_color = max(network_graph.node_renderer.data_source.data[color_by_this_attribute])
    maximum_value_color = min(network_graph.node_renderer.data_source.data[color_by_this_attribute])
    network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, minimum_value_color, maximum_value_color))

    # edge opacity and width
    network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

    plot.renderers.append(network_graph)

    show(plot)

## Функция для ipycytoscape 

In [58]:
def plot_cgraph(edges = edges_df, proteins = None, diseases = None, depth = 0):
    
    # функция строит граф ассоциация болезнь-белок
    # на вход подается таблица связей, таблица узлов, список белков и список болезней
    # список белков и список болезней - в формате списка []
    
    diseases_list = []
    classes_list = []
    table1 = pd.DataFrame()
    table2 = pd.DataFrame()
    table3 = pd.DataFrame()
    
    for item in diseases:
        if item in list(dis.diseaseName):
            diseases_list.append(item)
        if item in list(dis.disease):
            classes_list.append(item)
    
    if depth == 0:
        if proteins != None:
            proteins = pd.DataFrame(proteins)
            proteins.columns = ["source"]
            table1 = edges.merge(proteins, on = "source")
        
        if len(diseases_list) != 0:
            diseases_list = pd.DataFrame(diseases_list)
            diseases_list.columns = ["diseaseName"]
            table2 = edges.merge(diseases_list, on = "diseaseName")
            
        if len(classes_list) != 0:
            classes_list = pd.DataFrame(classes_list)
            classes_list.columns = ["target"]
            table3 = edges.merge(classes_list, on = "target")

        edges = concat(*[table1, table2, table3])
        edges = edges.drop_duplicates()
        # собственно, создаем граф    
        G = networkx.from_pandas_edgelist(edges, "source", "target")
            
    elif depth == 1:
        if proteins != None:
            proteins = pd.DataFrame(proteins)
            proteins.columns = ["source"]
            table1 = edges.merge(proteins, on = "source")
        if len(diseases_list) != 0:
            diseases_list = pd.DataFrame(diseases_list)
            diseases_list.columns = ["diseaseName"]
            table2 = edges.merge(diseases_list, on = "diseaseName")
        if len(classes_list) != 0:
            classes_list = pd.DataFrame(classes_list)
            classes_list.columns = ["target"]
            table3 = edges.merge(classes_list, on = "target")
        edges = concat(*[table1, table2, table3])
        edges = edges.drop_duplicates()
        # собственно, создаем граф
        G = networkx.from_pandas_edgelist(edges, "source", "diseaseName")
        
    classes_list = list(set(edges_df["target"]))
    diseases_list= list(set(edges_df["diseaseName"]))
    common_list = classes_list+diseases_list

    my_style = [{'selector': 'node','style': {'font-family': 'arial','font-size': '10px','label': 'data(id)','background-color': 'red'}}]

    for item in common_list:
        text = "node[id = \"" + item + "\"]"
        my_style.append({'selector': text,'style': {'font-family': 'arial','font-size': '10px','label': item,'background-color': 'green'}})

    ipycytoscape_obj = ipycytoscape.CytoscapeWidget()
    ipycytoscape_obj.graph.add_graph_from_networkx(G)
    ipycytoscape_obj.set_style(my_style)
    ipycytoscape_obj.set_style(my_style)
    return ipycytoscape_obj